In [626]:
%load_ext autoreload
%autoreload 2
    
import sys
import copy

sys.path.append("../lib/")

# from etna.experimental.classification.feature_extraction import TSFreshFeatureExtractor
import numpy as np
from tsfresh import extract_features, extract_relevant_features
from tsfresh.feature_extraction import settings
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier, Pool
from skimage.util.shape import view_as_windows
import scipy.linalg
from IPython.display import clear_output

tqdm.pandas()

import prepare_dataset
import feature_preparation

/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:362: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:362: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with th

In [ ]:
print

In [617]:
weather_train_path = "../data/weather_dataset.csv"

In [620]:
weather_train = pd.read_csv(weather_train_path, sep=";", index_col=0)

<ipython-input-1-3a06d231aa5a>:1: DtypeWarning: Columns (22,29) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_train = pd.read_csv(weather_train_path, sep=";", index_col=0)


In [624]:
weather_train.station_id.unique()

array([22550, 26063, 22820, 34009, 34123, 27947, 26898])

In [661]:
data_list = []
target_list = []

for key, data in tqdm(weather_train.groupby("station_id")):
    print(key)
    data_x, data_y = prepare_dataset.prepare_dataset(data)
    data_y["station_id"] = key

    data_list.append(data_x)
    target_list.append(data_y)
    
    clear_output(wait=True)

In [662]:
weather_train_x = pd.concat(data_list, ignore_index=True)
weather_train_y = pd.concat(target_list, ignore_index=True)

In [678]:
weather_train_y["local_time"] = weather_train_x.local_time

In [652]:
weather_train_y

,VETER,SHKVAL,METEL,DOZD,SNEG,GRAD,TUMAN,GOLOLED
0,False,False,False,True,True,False,False,False
1,False,False,False,True,True,False,False,False
2,False,False,False,True,True,False,False,False
3,False,False,False,True,True,False,False,False
4,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...
365164,False,False,False,False,True,False,False,False
365165,False,False,False,False,True,False,False,False
365166,False,False,False,False,True,False,False,False
365167,False,False,False,False,True,False,False,False


In [623]:
# weather_train_x, weather_train_y = prepare_dataset.prepare_dataset(weather_train)

  0%|          | 0/365169 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/365169 [00:00<?, ?it/s]

  0%|          | 0/365169 [00:00<?, ?it/s]

  0%|          | 0/365169 [00:00<?, ?it/s]

...

  0%|          | 0/365169 [00:00<?, ?it/s]

  0%|          | 0/365169 [00:00<?, ?it/s]

  0%|          | 0/365169 [00:00<?, ?it/s]

  0%|          | 0/365169 [00:00<?, ?it/s]

2022-12-11 19:10:34.767 | INFO     | prepare_dataset:prepare_dataset:96 - Current mode of preparation is processed.
2022-12-11 19:11:12.072 | INFO     | prepare_dataset:prepare_dataset:111 - Step: local_time_pipeline
2022-12-11 19:11:12.178 | INFO     | prepare_dataset:prepare_dataset:111 - Step: dd_pipeline
2022-12-11 19:11:20.349 | INFO     | prepare_dataset:prepare_dataset:111 - Step: ch_pipeline
2022-12-11 19:11:20.821 | INFO     | prepare_dataset:prepare_dataset:111 - Step: cm_pipeline
2022-12-11 19:11:21.230 | INFO     | prepare_dataset:prepare_dataset:111 - Step: cl_pipeline
2022-12-11 19:11:21.633 | INFO     | prepare_dataset:prepare_dataset:111 - Step: vv_pipeline
2022-12-11 19:11:22.177 | INFO     | prepare_dataset:prepare_dataset:114 - Filling nans!
2022-12-11 19:12:27.879 | INFO     | prepare_dataset:prepare_dataset:117 - Done (processed)!


In [636]:
one_station_x = weather_train_x[weather_train.station_id == 34123]
one_station_y = weather_train_y[weather_train.station_id == 34123]

In [213]:
one_station_y["local_time"] = one_station_x.local_time

<ipython-input-9-54dd8288e6e1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_station_y["local_time"] = one_station_x.local_time


In [214]:
one_station_x.sort_values("local_time", inplace=True)
one_station_y.sort_values("local_time", inplace=True)

<ipython-input-10-92cb5a27948e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_station_x.sort_values("local_time", inplace=True)
<ipython-input-10-92cb5a27948e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_station_y.sort_values("local_time", inplace=True)


In [430]:
cat_features = ['DD', 'N', 'Cl', 'Nh', 'H', 'Cm', 'Ch', 'VV', 'RRR', 'E', "E'"]
num_features = set(one_station.columns) - set(cat_features) - {"local_time", "station_id", "WW", "W1", "W2", "sss"}

In [431]:
one_station.head()

,local_time,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Ch,VV,Td,RRR,tR,E,Tg,E',sss,station_id
0,01.01.2006 00:00,-6.7,757.0,771.6,NaN,72.0,"Ветер, дующий с востока",5.0,NaN,NaN,...,"Перистые нитевидные, иногда когтевидные, не ра...",10.0,-10.9,None,NaN,None,NaN,None,None,34123
1,01.01.2006 03:00,-5.7,757.6,772.1,NaN,75.0,"Ветер, дующий с востоко-юго-востока",5.0,NaN,NaN,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",10.0,-9.5,None,NaN,None,NaN,None,None,34123
2,01.01.2006 06:00,-5.2,757.4,771.8,NaN,83.0,"Ветер, дующий с юго-востока",5.0,NaN,NaN,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",10.0,-7.7,None,NaN,None,NaN,None,None,34123
3,01.01.2006 09:00,-5.4,758.3,772.8,NaN,87.0,"Ветер, дующий с востока",4.0,NaN,NaN,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",8.0,-7.2,None,NaN,Cухая,NaN,Ровный слой сухого рассыпчатого снега покрывае...,30.0,34123
4,01.01.2006 12:00,-5.1,759.4,774.0,NaN,84.0,"Ветер, дующий с востока",5.0,NaN,NaN,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",18.0,-7.4,None,NaN,None,NaN,None,None,34123


In [432]:
one_station_x["dummy"] =1 

<ipython-input-29-9fe7e14ba09f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_station_x["dummy"] =1


In [436]:
settings_efficient = settings.MinimalFCParameters()
settings_efficient

{'sum_values': None, 'median': None, 'mean': None, 'length': None, 'standard_deviation': None, 'variance': None, 'root_mean_square': None, 'maximum': None, 'absolute_maximum': None, 'minimum': None}

In [437]:
ts = extract_features(one_station_x[["local_time", "dummy", "Pa"]], column_id="dummy", column_sort="local_time", default_fc_parameters=settings_efficient)

Feature Extraction: 100%|██████████| 1/1 [00:00<00:00, 52.61it/s]


In [438]:
ts

,Pa__sum_values,Pa__median,Pa__mean,Pa__length,Pa__standard_deviation,Pa__variance,Pa__root_mean_square,Pa__maximum,Pa__absolute_maximum,Pa__minimum
1,-908.3707,0.0,-0.017555,51743.0,0.866409,0.750665,0.866587,21.0,22.5,-22.5


In [ ]:
labels = one_station_y.columns

In [ ]:
united = pd.concat([one_station_x, one_station_y], axis=1)
for label in 
    united.rename({label: "feat_" + label for label in labels}, axis=1, inplace=True)


In [ ]:
y = one_station_y.iloc[1:].reset_index(drop=True)
x = united.iloc[:-1].reset_index(drop=True)

In [580]:
one_station_x = one_station_x.set_index("local_time")

In [584]:
one_station_x.sort_index(inplace=True)

In [587]:
one_station_x.head()

,T,Po,P,Pa,U,DD,Ff,ff10,ff3,N,...,Ch,VV,Td,RRR,tR,E,Tg,E',sss,station_id
local_time,,,,,,,,,,,,,,,,,,,,,
2005-01-02 03:00:00,-7.0,744.5,758.8,NaN,88.0,"Ветер, дующий с юго-востока",7.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",4.0,-8.7,NaN,NaN,NaN,NaN,NaN,NaN,34123
2005-01-02 06:00:00,-6.9,745.1,759.4,NaN,88.0,"Ветер, дующий с юго-востока",7.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",4.0,-8.5,NaN,NaN,NaN,NaN,NaN,NaN,34123
2005-01-02 09:00:00,-7.7,746.9,761.2,NaN,95.0,"Ветер, дующий с юго-юго-востока",5.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",2.4,-8.3,2.0,12.0,Cухая,NaN,Ровный слой сухого рассыпчатого снега покрывае...,35.0,34123
2005-01-02 12:00:00,-8.3,748.9,763.4,NaN,86.0,"Ветер, дующий с юго-востока",4.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",5.0,-10.2,NaN,NaN,NaN,NaN,NaN,NaN,34123
2005-01-02 15:00:00,-6.7,749.8,764.2,NaN,88.0,"Ветер, дующий с востоко-юго-востока",3.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",8.0,-8.3,NaN,NaN,NaN,NaN,NaN,NaN,34123


In [612]:
index_year_ago = one_station_x.index.intersection(one_station_x.index - pd.offsets.Day(365))

In [613]:
one_station_x.loc[index_year_ago]

,T,Po,P,Pa,U,DD,Ff,ff10,ff3,N,...,Ch,VV,Td,RRR,tR,E,Tg,E',sss,station_id
local_time,,,,,,,,,,,,,,,,,,,,,
2005-01-02 03:00:00,-7.0,744.5,758.8,NaN,88.0,"Ветер, дующий с юго-востока",7.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",4.0,-8.7,NaN,NaN,NaN,NaN,NaN,NaN,34123
2005-01-02 06:00:00,-6.9,745.1,759.4,NaN,88.0,"Ветер, дующий с юго-востока",7.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",4.0,-8.5,NaN,NaN,NaN,NaN,NaN,NaN,34123
2005-01-02 09:00:00,-7.7,746.9,761.2,NaN,95.0,"Ветер, дующий с юго-юго-востока",5.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",2.4,-8.3,2.0,12.0,Cухая,NaN,Ровный слой сухого рассыпчатого снега покрывае...,35.0,34123
2005-01-02 12:00:00,-8.3,748.9,763.4,NaN,86.0,"Ветер, дующий с юго-востока",4.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",5.0,-10.2,NaN,NaN,NaN,NaN,NaN,NaN,34123
2005-01-02 15:00:00,-6.7,749.8,764.2,NaN,88.0,"Ветер, дующий с востоко-юго-востока",3.0,NaN,NaN,100%.,...,"Перисто-кучевые одни или перисто-кучевые, сопр...",8.0,-8.3,NaN,NaN,NaN,NaN,NaN,NaN,34123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-11 09:00:00,0.7,748.3,762.2,-1.5,82.0,"Ветер, дующий с юго-запада",3.0,NaN,NaN,100%.,...,"Перистые нитевидные, иногда когтевидные, не ра...",10.0,-2.1,NaN,NaN,NaN,NaN,NaN,NaN,34123
2021-12-11 12:00:00,3.0,746.4,760.1,-1.9,80.0,"Ветер, дующий с юго-запада",3.0,10.0,10.0,100%.,...,NaN,10.0,-0.2,NaN,NaN,NaN,NaN,NaN,NaN,34123
2021-12-11 15:00:00,4.6,745.0,758.6,-1.4,77.0,"Ветер, дующий с западо-юго-запада",5.0,NaN,10.0,100%.,...,NaN,10.0,0.9,NaN,NaN,NaN,NaN,NaN,NaN,34123


In [359]:
def fast_rolling_window(s, window):
    s = s.values
    rolling = view_as_windows(s, window)
    first_row = rolling[0].astype(float)
    temp = np.triu(scipy.linalg.toeplitz(first_row)).T
    temp[np.triu_indices(3, 1)] = np.nan
    temp = np.fliplr(temp)[:-1]

    return np.vstack([temp, rolling])

In [673]:
weather_train_x

,local_time,T,Po,P,Pa,U,Ff,Cl,Cm,Ch,...,ff3_isnan,ff3_val,ff10_isnan,ff10_val,N_isnan,N_val,Nh_isnan,Nh_val,H_isnan,H_val
0,2022-12-09 21:00:00,-7.5,765.6,766.3,1.2,91.0,1.0,Кучево-дождевые-волокнистые,Нет данных!,Нет данных!,...,True,1.0,True,1.0,False,100.0,False,100.0,False,250.0
1,2022-12-09 18:00:00,-7.8,764.4,765.2,0.8,90.0,2.0,Кучево-дождевые-волокнистые,Нет,Нет,...,True,2.0,True,2.0,False,95.0,False,95.0,False,450.0
2,2022-12-09 15:00:00,-8.1,763.6,764.5,0.7,89.0,2.0,Кучево-дождевые-волокнистые,Высококучевые-дождь,Нет,...,True,2.0,True,2.0,False,95.0,False,75.0,False,800.0
3,2022-12-09 12:00:00,-8.3,762.9,763.7,0.8,90.0,2.0,Кучево-дождевые-волокнистые,Нет,Нет,...,True,2.0,True,2.0,False,95.0,False,95.0,False,450.0
4,2022-12-09 09:00:00,-8.8,762.1,762.8,0.7,90.0,2.0,Слоистые-разорванные-не-плохая,Нет данных!,Нет данных!,...,True,2.0,True,2.0,False,100.0,False,100.0,False,450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365164,2005-02-01 15:00:00,-6.7,749.8,764.2,0.8,88.0,3.0,Слоистые-разорванные-плохая-погода,Высокослоистые-просвет,Перисто-кучевые+,...,True,3.0,True,3.0,False,100.0,False,100.0,False,450.0
365165,2005-02-01 12:00:00,-8.3,748.9,763.4,2.2,86.0,4.0,Слоистые-разорванные-плохая-погода,Высокослоистые-просвет,Перисто-кучевые+,...,True,4.0,True,4.0,False,100.0,False,100.0,False,250.0
365166,2005-02-01 09:00:00,-7.7,746.9,761.2,1.8,95.0,5.0,Слоистые-разорванные-плохая-погода,Высокослоистые-просвет,Перисто-кучевые+,...,True,5.0,True,5.0,False,100.0,False,100.0,False,250.0
365167,2005-02-01 06:00:00,-6.9,745.1,759.4,0.6,88.0,7.0,Слоистые-разорванные-плохая-погода,Высокослоистые-просвет,Перисто-кучевые+,...,True,7.0,True,7.0,False,100.0,False,100.0,False,450.0


In [647]:
numeric_features = []
categorial_features = []


for col in tqdm(one_station_x.columns):
    if col in ["local_time", "station_id"]:
        continue
    try:
        one_station_x[col].mean()
        numeric_features.append(col)
    except:
        categorial_features.append(col)

  0%|          | 0/38 [00:00<?, ?it/s]

In [648]:
numeric_features

['T',
 'Po',
 'P',
 'Pa',
 'U',
 'Ff',
 'VV',
 'Td',
 'tR',
 'sss',
 'dd_isnan',
 'dd_changed',
 'dd_x_rad',
 'dd_y_rad',
 'Tn_isnan',
 'Tn_val',
 'Tx_isnan',
 'Tx_val',
 'Tg_isnan',
 'Tg_val',
 'ff3_isnan',
 'ff3_val',
 'ff10_isnan',
 'ff10_val',
 'N_isnan',
 'N_val',
 'Nh_isnan',
 'Nh_val',
 'H_isnan',
 'H_val']

In [650]:
categorial_features

['Cl', 'Cm', 'Ch', 'RRR', 'E', "E'"]

In [681]:
from loguru import logger


import scipy.stats as sps


def mad(data, axis=None):
    return np.mean(np.abs(data - np.mean(data, axis, keepdims=True)), axis)


def count_time_no_event(observation: pd.Series):
    observation = ~observation
    tmp = observation.cumsum() - observation.cumsum().where(~observation).ffill().fillna(0).astype(int)
    return tmp.shift(1, fill_value=0)


def get_window_features(df, features, window=8):
    info_rolling = {}
    for feature in features:
        rolling_window = fast_rolling_window(df[feature], window=window)
        info_rolling.update({
            f"{feature}_mean_window={window}": np.mean(rolling_window, axis=-1),
            f"{feature}_median_window={window}": np.median(rolling_window, axis=-1),
            f"{feature}_std_window={window}": np.std(rolling_window, axis=-1, ddof=1),
            f"{feature}_mad_window={window}": mad(rolling_window, axis=-1),
            f"{feature}_skew_window={window}": sps.skew(rolling_window, axis=-1),
            f"{feature}_kurt_window={window}": sps.kurtosis(rolling_window, axis=-1),
            f"{feature}_min_window={window}": np.min(rolling_window, axis=-1),
            f"{feature}_max_window={window}": np.max(rolling_window, axis=-1),
            f"{feature}_count_window={window}": np.count_nonzero(~np.isnan(rolling_window), axis=-1),
            f"{feature}_has_nan_window={window}": np.any(np.isnan(rolling_window), axis=-1).astype(int)
        })

    return pd.DataFrame(info_rolling, index=df.index).fillna(0)


def get_window_features_with_offset(df, target, features, offset):
    index = df.index
    index_year_ago = index.intersection(index - offset)

    target_year_ago = target.loc[index_year_ago]
    target.index += pd.offsets.Day(365)
    df_new = df.merge(target_year_ago, how="left", on="local_time", suffixes=(None, "_year_ago"))

    return df_new


def make_features_for_station(df, targets, numeric_features, categorial_columns, windows=None):
    df.sort_values("local_time", inplace=True)
    targets.sort_values("local_time", inplace=True)

    windows = windows or [8, 80, 360]
    numeric_features = copy.copy(numeric_features)

    new_df = df[numeric_features + categorial_columns + ["station_id", "local_time"]]

    # представим дату как синус и косинус
    logger.info("Coding date as (sin, cos)")
    new_df["sin_day_of_year"] = new_df.local_time.map(lambda x: np.sin(2 * np.pi * x.timetuple().tm_yday / 365))
    new_df["cos_day_of_year"] = new_df.local_time.map(lambda x: np.cos(2 * np.pi * x.timetuple().tm_yday / 365))
    new_df["sin_time_of_day"] = new_df.local_time.map(lambda x: np.sin(2 * np.pi * x.timetuple().tm_hour / 24))
    new_df["cos_time_of_day"] = new_df.local_time.map(lambda x: np.cos(2 * np.pi * x.timetuple().tm_hour / 24))

    # считаем кол-во дней без дождя, снега и тд
    logger.info("Counting time without events")
    for col in tqdm(targets.columns):
        if col in ["station_id", "local_time"]:
            continue
        new_feature_name = f"no_{col}_time"
        new_df[new_feature_name] = count_time_no_event(targets[col].shift().fillna(False))
        numeric_features.append(new_feature_name)

    new_numeric_df = []
    logger.info("Window features")
    for window in tqdm(windows):
        new_numeric_df.append(
            get_window_features(
                new_df, numeric_features, window
            )
        )
    new_df = pd.concat(new_numeric_df + [new_df], axis=1)

    return new_df.iloc[:-1], targets.iloc[1:]

In [682]:
prepared_data_list = []
prepared_target_list = []

for key, data in tqdm(weather_train_x.groupby("station_id")):
    one_station_y = weather_train_y[weather_train_y.station_id == key] 

    prodata = make_features_for_station(data, one_station_y, numeric_features, categorial_features, [8, 80, 360])

    prepared_data_list.append(data_x)
    prepared_target_list.append(data_y)
    
    clear_output(wait=True)

In [684]:
all_train = pd.concat(prepared_data_list, ignore_index=True)
all_target = pd.concat(prepared_target_list, ignore_index=True)

In [688]:
all_train.to_csv("../data/my_ready_dataset.csv", index=False)

In [687]:
all_target.to_csv("../data/my_ready_targets.csv", index=False)